<a href="https://colab.research.google.com/github/gonzalesricardo/PARADIGMAS-DE-LINGUAGEM-DE-PROGRAMA-O/blob/main/RNG_Gapminder_WID.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<head>
  <meta name="author" content="Rogério de Oliveira">
  <meta institution="author" content="Universidade Presbiteriana Mackenzie">
</head>

<img src="http://meusite.mackenzie.br/rogerio/mackenzie_logo/UPM.2_horizontal_vermelho.jpg" width=300, align="right"> 

<h1 align=left><font size = 8, style="color:rgb(200,0,0)"><b>Ciência de Dados </b></font></h1> 
<h1 align=left><font size = 5, style="color:rgb(0,0,0)"><b>Paradigmas de Linguagens de
Programação para Ciência de Dados</b></font></h1> 

*Rogério de Oliveira* 

<br>
<br>

---


Este é um material de apoio para Atividade de Aprofundamento 2. Ele mostra como obter e tratar dados do [Gapminder](https://www.gapminder.org/) e do [WID](https://wid.world/). Você pode empregar uma ou ambas as fontes de dados. 

# Gapminder

![imagem](https://www.gapminder.org/wp-content/themes/gapminder2/images/gapminder-logo.svg)

Essa é uma importante fonte de dados aberta que contêm diversas informações e índices relacionados ao desenvolvimento dos países.

Acesse [aqui](https://www.gapminder.org/data/) para extrair os dados. 

Escolha os dados de seu interesse. Faça o download no formato `.csv` para o local que desejar.

In [51]:
import pandas as pd

ool = pd.read_csv('https://drive.google.com/file/d/1K7iRLQOeNZADklJqRu6HwbRx1pBsYnE0/view?usp=sharing')
ool.head()


ParserError: ignored

# Preparação dos Dados

Vamos obter aqui dados de escolaridade e emissões de co2 do Brasil. O Gapminder fornece esses dados em conjuntos separados e vamos combinar esses dados para nossa análise.

In [ ]:
school_BR = school[ school.country == 'Brazil' ]
school_BR

In [ ]:
BR = pd.melt(school_BR, id_vars=['country'])
BR.head()

In [ ]:
BR = BR.rename(columns={'variable':'year','value':'school'})
BR.head()

In [ ]:
co2 = pd.read_csv('http://meusite.mackenzie.br/rogerio/data_load/gapminder_co2_emissions_tonnes_per_person.csv')
co2_BR = co2[ co2.country == 'Brazil' ]
co2_BR

In [ ]:
co2_BR = pd.melt(co2_BR, id_vars=['country'])
co2_BR = co2_BR.rename(columns={'variable':'year','value':'co2'})
co2_BR.head()

In [ ]:
BR = pd.merge(BR,co2_BR,on=['country','year'])
BR.head()

# Visualização e Análise dos Dados

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 

In [ ]:
plt.figure(figsize=(12,6))

sns.lineplot(x=BR.year, y=BR.school, label='school')
sns.lineplot(x=BR.year, y=BR.co2, label='co2')

plt.title('Brasil: Escolaridade X Emissões CO2', fontsize=14)
plt.legend()
plt.xticks(rotation=90)
plt.show()

In [ ]:
import numpy as np 
plt.figure(figsize=(12,6))

sns.lineplot(x=BR.year, y=np.log( BR.school ), label='school')
sns.lineplot(x=BR.year, y=np.log( BR.co2 ), label='co2')

plt.title('Brasil: Escolaridade X Emissões CO2 (Log Scale)', fontsize=14)
plt.legend()
plt.xticks(rotation=90)
plt.show()

# World Inequality Database

![imagem](https://www.hup.harvard.edu/images/jackets/9780674979857.jpg)

Vamos ver como combinar dados de outras fontes? 

O World Inequality Database é uma base de dados aberta que mantem informações sobre desigualdade e concentração de renda no mundo. Esses dados são a base do livro **Capital in the Twenty-First Century** de *Thomas Piketty*.

In [ ]:
from IPython.display import IFrame
IFrame('https://wid.world/data/', width='1000', height=400)

Se você for empregar essa base escolha os índices de interesse, a estrutura da tabela e dê preferência para o formato `.xlsx` para download.

# Preparação dos Dados

Vamos selecionar dados do Brasil de 2000-2019 sobre a renda per capta (gpd) e o percentual da renda concentrado nos 10% mais ricos da população (percentil 10), e combinar esses dados com as informações que já coletamos do gapminder.

In [ ]:
gpd_BR = pd.read_excel('http://meusite.mackenzie.br/rogerio/data_load/WID_Data_13092021-135730.xlsx',header=None)
gpd_BR


In [ ]:
gpd_BR.columns = ['country', 'ind_description',  'ind_code', 'year', 'value']
gpd_BR.year = gpd_BR.year.astype(str)
display(gpd_BR)
display(gpd_BR.dtypes)
        

In [ ]:
gpd_BR_all = gpd_BR[ gpd_BR.ind_code == 'pall' ][['country', 'year', 'value']]
gpd_BR_perc = gpd_BR[ gpd_BR.ind_code == 'p90p100' ][['country', 'year', 'value']]

display(gpd_BR_all)
display(gpd_BR_perc)

In [ ]:
BR = pd.merge(BR,gpd_BR_all,on=['country','year'])
BR = BR.rename(columns={'value':'gpd'})
BR.head()

In [ ]:
BR = pd.merge(BR,gpd_BR_perc,on=['country','year'])
BR = BR.rename(columns={'value':'gpd_perc10'})
BR.head()

# Visualização e Análise dos Dados

Como queremos comparar dados em escalas muito diferentes uma sugestão é empregarmos dados normalizados.

In [ ]:
for c in BR[['school','co2','gpd','gpd_perc10']]:
  BR[c] = BR[c] / BR[c].max()
BR.head()

In [ ]:
plt.figure(figsize=(12,6))

for c in BR[['school','co2','gpd','gpd_perc10']]:
  sns.lineplot(x=BR.year, y=BR[c], label=c)

plt.title('Brasil, Escolaridade x CO2 X Renda X Concentração', fontsize=14)
plt.legend()
plt.xticks(rotation=90)
plt.show()

# Conclusões

Às análises devem ser finalizadas com uma conclusão, em geral um texto escrito e que inclui os resultados, limitações da análise e sugestões de próximos passos.

> *Os dados acima mostram que a escolaridade, renda e concentração de renda vem crescendo no Brasil de modo bastante correlacionado nas últimas décadas, havendo entretanto um maior aumento de emissões nos últimos anos.*